In [3]:
import Bio
from Bio import SeqIO
import pandas as pd 
import numpy as np 
import re
import sys
import os
import os.path
import datetime

"""
function that identifies the position of stretches of N in a sequence

find the start and end position of a stretch of N for a sequence
parameters are;
motif (number of consecutive N from the user input)
sequence (sequence to look into)
"""
def find_stretch_N(motif, sequence):
    df_pos_N=pd.DataFrame(columns=['start', 'end'])
    #find the occurence of the motif in the sequence, 
    #once a motif is found, the algorythm starts at the end of the motif
    str_sequence=str(sequence.seq)
    end_last_stretch=-1
    for match in re.finditer(motif, str_sequence):
        start_pos = match.start() 
        
        #start at the end of last stretch
        if (start_pos>end_last_stretch):
            end_pos = match.end() 
            #more than X consecutive N
            while(end_pos<len(sequence) and sequence[end_pos]=="N" ):
                end_pos+=1
            end_last_stretch=end_pos
            #append the dataframe of N stretches
            df_pos_N=df_pos_N.append({'start':start_pos, 'end':end_pos},ignore_index=True)

    return df_pos_N


        
""" 
function that return the list of flagged SNP whithin a list of SNP position or whithin the whole sequence

find the SNP around the stretches of N that are in the interval
parameters are;
nuc_interval (number of snp around stretch)
df_pos_N (data frame of start-end of stretch of N for a sequence)
list_SNP (list of position of SNP from a VCF in bed format)
sequence (sequence to look into)
"""
def find_mutation_not_SNP(nuc_interval, df_pos_N, list_SNP, reference, sequence):
        
    #list of flagged position to return
    df_flag=pd.DataFrame(columns=['sequence id', 'pos', 'dist from N', 'protected', 'nucleotide flag'])
    #creates a df for sequence and their respective number of flagged sites
    df_nb_flag=pd.DataFrame(columns=['sequence id', 'number of flagged sites'])   
    nb_flag=0
    
    #sequence to look at in list format
    print(sequence.id)
    list_seq=list(sequence.seq)
    
    #loop through the rows of the stretch of N
    for index, row in df_pos_N.iterrows():
               
        start_N=row['start']
        end_N=row['end']
        i=1
        
        #get the Y number nucleotides before and after
        while (i<=nuc_interval):
            
            pos_before=start_N - i
            pos_after=end_N + i
            
            #if the pos_before is before the beginning of the sequence
            if(pos_before>=0):
                #nuc at the position
                nuc_before=list_seq[pos_before]
            
                #compare to the reference sequence to see if this position is mutated
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_before!=reference[pos_before] and nuc_before!="N"):
                    if(pos_before in list_SNP):
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_before},ignore_index=True)
                        nb_flag+=1
                        
            #if the pos_end is after the end of the sequence
            if(pos_after<len(sequence)):
                
                #nuc at the position
                nuc_after=list_seq[pos_after]
                
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_after!=reference[pos_after] and nuc_after!="N"): 
                    if(pos_after in list_SNP):
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_after},ignore_index=True)
                        nb_flag+=1
            
            #iterate
            i+=1
        
        #return a list of the sorted position of the SNP flagged and remove duplicates
        sorted_flag=df_flag.sort_values(by=['pos'])
        sorted_flag.drop_duplicates(subset =['sequence id', 'pos'], keep = 'first', inplace = True) 
        
    df_nb_flag=df_nb_flag.append({'sequence id':sequence.id, 'number of flagged sites':nb_flag}, ignore_index=True)
    
    return sorted_flag, df_nb_flag



"""
function return the list of flagged SNP that aren't in the list of protected positions
it also returns the list of flagged SNP that are in the list of protected positions for statistics

parameters are;
nuc_interval (number of snp around stretch)
df_pos_N (data frame of start-end of stretch of N for a sequence)
list_protected_SNP (list of position of SNP from a VCF in bed format that are correct/protected)
sequence (sequence to look into)

"""
def find_mutation_not_protected_SNP(nuc_interval, df_pos_N, df_protected_SNP, reference, sequence):
    
    list_protected_SNP=df_protected_SNP['START'].tolist()
    
    #list of flagged position to return
    df_flag=pd.DataFrame(columns=['sequence id', 'pos', 'dist from N', 'protected', 'nucleotide flag'])
    #list of protected SNP that are close to the stretch of N
    protected_flag=[]
    print(sequence.id)
    #creates a df for sequence and their respective number of flagged sites
    df_nb_flag=pd.DataFrame(columns=['sequence id', 'number of flagged sites'])
    #count the number of flagged position for this sequence
    nb_flag=0
    
    #sequence to look at in list format
    list_seq=list(sequence.seq)
    
    #loop through the rows of the stretch of N
    for index, row in df_pos_N.iterrows():
        
        start_N=row['start']
        end_N=row['end']
        i=1
        
        #get the Y number nucleotides before and after
        while (i<=nuc_interval):
            
            pos_before=start_N - i
            pos_after=end_N + i
            
            #if the pos_before is before the beginning of the sequence
            if(pos_before>=0):
                #nuc at the position
                nuc_before=list_seq[pos_before]
            
                #compare to the reference sequence to see if this position is mutated
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_before!=reference[pos_before] and nuc_before!="N"):
                    if(pos_before in list_protected_SNP):
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':1, 'nucleotide flag':nuc_before},ignore_index=True)
                        protected_flag.append(pos_before+1)
                    else:
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_before},ignore_index=True)
                        nb_flag+=1
            
            
            #if the pos_end is after the end of the sequence
            if(pos_after<len(sequence)):
                
                #nuc at the position
                nuc_after=list_seq[pos_after]
        
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_after!=reference[pos_after] and nuc_after!="N"): 
                    if(pos_after in list_protected_SNP):
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':1, 'nucleotide flag':nuc_after},ignore_index=True)
                        protected_flag.append(pos_after+1)
                    else:
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_after},ignore_index=True)
                        nb_flag+=1

            
            
            #iterate
            i+=1
        
        #return a list of the sorted position of the SNP flagged and protected, also remove duplicates
        sorted_flag=df_flag.sort_values(by=['pos'])
        sorted_flag.drop_duplicates(subset =['sequence id', 'pos'], keep = 'first', inplace = True) 
        protected_flag=list(set(protected_flag))
        protected_flag=sorted(protected_flag)
            
    df_nb_flag=df_nb_flag.append({'sequence id':sequence.id, 'number of flagged sites':nb_flag},ignore_index=True)
            
    return sorted_flag, protected_flag, df_nb_flag
    
    
    
"""
function return the list of flagged SNP that aren't in the list of protected positions and are different than the specified allele
it also returns the list of flagged SNP that are in the list of protected positions for statistics

parameters are;
nuc_interval (number of snp around stretch)
df_pos_N (data frame of start-end of stretch of N for a sequence)
list_protected_SNP (list of position of SNP from a VCF in bed format that are correct/protected)
sequence (sequence to look into)
"""   
def find_mutation_not_protected_allele_SNP(nuc_interval, df_pos_N, df_protected_SNP, reference, sequence):
        
    list_protected_SNP=df_protected_SNP['START'].tolist()
    #list of flagged position to return
    df_flag=pd.DataFrame(columns=['sequence id', 'pos', 'dist from N', 'protected', 'nucleotide flag'])
    #list of protected SNP that are close to the stretch of N
    protected_flag=[]
    
    #creates a df for sequence and their respective number of flagged sites
    df_nb_flag=pd.DataFrame(columns=['sequence id', 'number of flagged sites'])
    #count the number of flagged position for this sequence
    nb_flag=0
    
    #sequence to look at in list format
    list_seq=list(sequence.seq)
    
    #loop through the rows of the stretch of N
    for index, row in df_pos_N.iterrows():
        
        start_N=row['start']
        end_N=row['end']
        i=1
        
        #get the Y number nucleotides before and after
        while (i<=nuc_interval):
            
            pos_before=start_N - i
            pos_after=end_N + i
            
            #if the pos_before is before the beginning of the sequence
            if(pos_before>=0):
                #nuc at the position
                nuc_before=list_seq[pos_before]
            
                #compare to the reference sequence to see if this position is mutated
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_before!=reference[pos_before] and nuc_before!="N"):
                    #allele specific protection                          
                    if(pos_before in list_protected_SNP):
                        row_pos = df_protected_SNP.loc[df_protected_SNP['START'] == pos_before]
                        row_pos_allele = row_pos['MAJ_ALLELE']
                        #if it is corresponding to the alternate allele either
                        if(str(row_pos_allele)!=nuc_before):
                            df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':2, 'nucleotide flag':nuc_before},ignore_index=True)
                            protected_flag.append(pos_before+1)
                            #nb_flag+=1
                        else:
                            df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':1, 'nucleotide flag':nuc_before},ignore_index=True)
                            protected_flag.append(pos_before+1)
                            #nb_flag+=1
                    else:
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_before+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_before},ignore_index=True)
                        nb_flag+=1
            
            
            #if the pos_end is after the end of the sequence
            if(pos_after<len(sequence)):
                
                #nuc at the position
                nuc_after=list_seq[pos_after]
        
                #if the pos_end is after the end of the sequence or pos_before is before the start
                #add the pos in the list of flag SNP if they aren't a SNP position
                if(nuc_after!=reference[pos_after] and nuc_after!="N"): 
                    #if(pos_after not in list_protected_SNP):
                    if(pos_after in list_protected_SNP):
                        row_pos = df_protected_SNP.loc[df_protected_SNP['START'] == pos_after]
                        row_pos_allele = row_pos['MAJ_ALLELE']
                        #if it is corresponding to the alternate allele either
                        if(str(row_pos_allele)!=nuc_after):
                            df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':2, 'nucleotide flag':nuc_after},ignore_index=True)
                            protected_flag.append(pos_after+1)
                            #nb_flag+=1
                        else:
                            df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':1, 'nucleotide flag':nuc_after},ignore_index=True)
                            protected_flag.append(pos_after+1)
                            #nb_flag+=1
                    else:
                        df_flag=df_flag.append({'sequence id':sequence.id, 'pos':(pos_after+1), 'dist from N':i, 'protected':0, 'nucleotide flag':nuc_after},ignore_index=True)
                        nb_flag+=1
            #iterate
            i+=1
        
        #return a list of the sorted position of the SNP flagged and protected, also remove duplicates
        sorted_flag=df_flag.sort_values(by=['pos'])
        sorted_flag.drop_duplicates(subset =['sequence id', 'pos'], keep = 'first', inplace = True) 
        protected_flag=list(set(protected_flag))
        protected_flag=sorted(protected_flag)
            
    df_nb_flag=df_nb_flag.append({'sequence id':sequence.id, 'number of flagged sites':nb_flag},ignore_index=True)
            
    return sorted_flag, protected_flag, df_nb_flag
    
    
    
"""
function returns the current sequence with the flagged SNP masked (replace by N) in the sequence

parameters are;
flag_SNP list of SNP that are flagged
sequence current fasta sequence in analysis
"""
def out_fasta(flag_SNP, sequence):
    
    str_sequence=str(sequence.seq)
    #return to a 0 based list
    flag_SNP.pos=(flag_SNP.pos-1)
    #replace the flaged SNP by N
    for index, row in flag_SNP.iterrows():
        pos=row['pos']
        #do not replace the protected SNP 
        if(row['protected']==0):
            str_sequence=str_sequence[:pos]+"N"+str_sequence[pos+1:]
            
    return str_sequence
    
    
"""
function that writes the dataframe of SNP that were flag and if they are protected or not

parameters are;
flag_SNP dataframe and output file name
"""
def write_flag(output_file, flagged_SNP):
    
    #return to a 1 based sequence
    flagged_SNP.pos=(flagged_SNP.pos+1)
    form='%s', '%d', '%d', '%d', '%s'
    f=open(output_file+'.txt','ab')
    np.savetxt(f, flagged_SNP, fmt=form, delimiter="\t")
    f.close()
    
    
"""
function write in a file the current sequence with the flagged SNP masked (replace by N) in the sequence

parameters are;
flag_SNP list of SNP that are flagged
Sequence that is currently beeing analyse
sequence with replacement from function out_fasta
"""
def write_fasta(output_file, sequence, masked_sequence):
    
    #write the fasta file with the masked position
    with open(output_file+".fasta", "a") as out_fasta_file:
        out_fasta_file.write(">"+sequence.id + "\n" + masked_sequence + "\n")
        
"""
function that writes the stats for the position that were protected in an output file

parameters are;
flagged_protected_SNP list of SNP that are flagged and protected
sequence current fasta sequence in analysis
name of the output file
"""
def write_protected_stats(output_file, sequence, flagged_protected_SNP):
    
    with open(output_file+"protected.stats", "a") as stats_file:
        stats_file.write(sequence.id+ " as "+ str(len(flagged_protected_SNP))+ "protected SNP within the interval to the stretches of N.\n")
        for pos in flagged_protected_SNP:
            stats_file.write(sequence.id + "\t")
            stats_file.write("%s\n" % pos)

"""
function that writes the number of SNP flag in each sequence in a file

parameters are;
df_nb_flag number of SNP flag per sequence (sequence id, number of snp flag)
name of the output file
"""            
def write_nb_flag_per_seq(output_file, df_nb_flag):

    form='%s', '%d'
    f=open(output_file+'_nb_flag_per_seq.txt','ab')
    np.savetxt(f, df_nb_flag, fmt=form, delimiter="\t")
    f.close()
    
    
#MAIN
def main():
    
    number_N=int(input("Minimum number of consecutive N: "))
    nuc_interval=int(input("Interval of nucleotide to look at around the stretch of N: "))

    motif="N" * number_N
    print("motif to find "+motif)

    #fasta of the alignement
    fasta_file=input("path and filename for the alignement file in this format: USER/EXEMPLE/file.fasta ->")
    records = list(SeqIO.parse(fasta_file, "fasta"))
    
    #Reference fasta file
    reference_file=input("path and filename for the reference sequence file in this format: USER/EXEMPLE/reference.fasta ->")
    reference_fasta = list(SeqIO.parse(reference_file, "fasta"))
    reference_seq=list(reference_fasta[0].seq)
    
    #bed file of the positions of the SNPs
    bed_file=input("path and filename for the bed file of all SNP in this format: USER/EXEMPLE/file.bed \n leave empty if you want the program to look at every positions ->")
    if(bed_file!=""):
        df_SNP=pd.read_table(bed_file, sep='\t', header=None)
        df_SNP.columns = ['CHROM', 'START', 'END', 'MAJ_ALLELE']
        list_SNP=df_SNP['START'].tolist()
        fct="SNP"
    else:
        fct=""
        
    protected_file=input("path and filename for the bed of protected SNP file in this format: USER/EXEMPLE/protected.bed \n leave empty if you don't have a list of SNP that you want to protect during the analyses ->")
    if(protected_file!=""):
        df_protected_SNP=pd.read_table(protected_file, sep='\t', header=None)
        if(len(df_protected_SNP.columns)==3):
            df_protected_SNP.columns = ['CHROM', 'START', 'END']
            fct="protected"
        if(len(df_protected_SNP.columns)==4):
            df_protected_SNP.columns = ['CHROM', 'START', 'END', 'MAJ_ALLELE']
            fct="protected_allele"
        if(len(df_protected_SNP.columns)>4):
            print ("Invalid number of columns in protected bed file. Expected 3 or 4 column ('CHR', 'START', 'END', *'Major allele'*)")
            sys.exit(0)
    if(bed_file=="" and protected_file==""):
        fct=""
    if(bed_file!="" and protected_file!=""):
        print("*************You can not give a list of SNP and a list of SNP to protect*************")
        sys.exit(0)
    
    
    #name the output file
    output_file=input("path and filename for the output prefix  in this format: USER/EXEMPLE/output ->")
    
    
    #time 
    begin_time = datetime.datetime.now()
    
    
    #check if a file doesn't already exists and if it does, it deletes it
    if(os.path.isfile(output_file + ".txt")):
        print("\n")
        print("*************File "+ output_file + ".txt" " already exist. File was deleted*************")
        print("\n")
        #sys.exit(0)
        os.remove(output_file+".txt")
    if(os.path.isfile(output_file + ".fasta")):
        print("\n")
        print("*************File "+ output_file + ".fasta" + " already exist. File was deleted*************")
        print("\n")
        #sys.exit(0)
        os.remove(output_file+".fasta")
    if(os.path.isfile(output_file + ".stats")):
        print("\n")
        print("*************File "+ output_file + ".stats"+ " already exist. File was deleted*************")
        print("\n")
        #sys.exit(0)
        os.remove(output_file + ".stats")
    if(os.path.isfile(output_file + "_nb_flag_per_seq.txt")):
        print("\n")
        print("*************File "+ output_file + "_nb_flag_per_seq.txt"+ " already exist. File was deleted*************")
        print("\n")
        #sys.exit(0)
        os.remove(output_file + "_nb_flag_per_seq.txt")
    
    #loop through all the sequences in the fasta file 
    for sequence in records:
        
        #check if the sequence has the right length or stop
        if(len(sequence.seq)!= 29903):
            print("*************               ERROR : sequence ", sequence.id, " has the wrong lenght ( reference = 29903).          ******************")
            sys.exit(0)
        
        
        #find the stretch of at least X (parameter) N.
        pos_stretch=find_stretch_N(motif, sequence)
        #make sure the dataframe isn't empty
        if(pos_stretch.empty):
            flagged_SNP=pd.DataFrame(columns=['sequence id', 'pos', 'dist from N', 'protected', 'nucleotide flag'])
            df_nb_flag=pd.DataFrame(columns=['sequence id', 'number of flagged sites'])
            df_nb_flag=df_nb_flag.append({'sequence id':sequence.id , 'number of flagged sites': 0},ignore_index=True)
            masked_sequence=out_fasta(flagged_SNP, sequence)
            write_fasta(output_file, sequence, masked_sequence)
            write_nb_flag_per_seq(output_file, df_nb_flag)
            
        else:
            #find the SNP that are flagged for this sequence
            if(fct=="protected"):
                flagged_SNP, flagged_protected_SNP, df_nb_flag=find_mutation_not_protected_SNP(nuc_interval, pos_stretch, df_protected_SNP, reference_seq, sequence)
                write_protected_stats(output_file, sequence, flagged_protected_SNP)
            if(fct=="protected_allele"):
                flagged_SNP, flagged_protected_SNP, df_nb_flag=find_mutation_not_protected_allele_SNP(nuc_interval, pos_stretch, df_protected_SNP, reference_seq, sequence)
                write_protected_stats(output_file, sequence, flagged_protected_SNP)
            if(fct=="SNP"):
                flagged_SNP, df_nb_flag=find_mutation_not_SNP(nuc_interval, pos_stretch, list_SNP, reference_seq, sequence)
            if(fct==""):
                list_SNP_all=range(len(reference_seq))
                flagged_SNP, df_nb_flag=find_mutation_not_SNP(nuc_interval, pos_stretch, list_SNP_all, reference_seq, sequence)
            

            masked_sequence=out_fasta(flagged_SNP, sequence)
            #write the fasta sequences with masked SNP
            write_fasta(output_file, sequence , masked_sequence)
            #write the results into a file
            write_flag(output_file, flagged_SNP) 
            write_nb_flag_per_seq(output_file, df_nb_flag)
            
        
    #time it has taken
    total_time=(datetime.datetime.now() - begin_time)
    print("execution time: " + str(total_time))
        

if __name__ == "__main__":
    main()
    
    

Minimum number of consecutive N: 3
Interval of nucleotide to look at around the stretch of N: 5
motif to find NNN
path and filename for the alignement file in this format: USER/EXEMPLE/file.fasta ->modified_fasta_july_9.fasta
path and filename for the reference sequence file in this format: USER/EXEMPLE/reference.fasta ->sequence.fasta
path and filename for the bed file of all SNP in this format: USER/EXEMPLE/file.bed 
 leave empty if you want the program to look at every positions ->
path and filename for the bed of protected SNP file in this format: USER/EXEMPLE/protected.bed 
 leave empty if you don't have a list of SNP that you want to protect during the analyses ->SNP_18_to_protect.bed
path and filename for the output prefix  in this format: USER/EXEMPLE/output ->july_9_3N_5
hCoV_19_Algeria_G0638_2264_2020_EPI_ISL_418241_2020_03_02
hCoV_19_Algeria_G0640_2265_2020_EPI_ISL_418242_2020_03_08
hCoV_19_Algeria_G0860_2262_2020_EPI_ISL_420037_2020_03_02
hCoV_19_Argentina_C121_2020_EPI_ISL

hCoV_19_Australia_NSW18_2020_EPI_ISL_417384_2020_03_05
hCoV_19_Australia_NSW190_2020_EPI_ISL_427736_2020_03_16
hCoV_19_Australia_NSW191_2020_EPI_ISL_427786_2020_03_25
hCoV_19_Australia_NSW192_2020_EPI_ISL_427797_2020_03_26
hCoV_19_Australia_NSW193_2020_EPI_ISL_427798_2020_03_20
hCoV_19_Australia_NSW194_2020_EPI_ISL_427763_2020_03_23
hCoV_19_Australia_NSW195_2020_EPI_ISL_427767_2020_03_22
hCoV_19_Australia_NSW196_2020_EPI_ISL_427806_2020_03_26
hCoV_19_Australia_NSW197_2020_EPI_ISL_427746_2020_03_21
hCoV_19_Australia_NSW198_2020_EPI_ISL_427707_2020_03_23
hCoV_19_Australia_NSW199_2020_EPI_ISL_427747_2020_03_22
hCoV_19_Australia_NSW19_2020_EPI_ISL_417385_2020_03_06
hCoV_19_Australia_NSW200_2020_EPI_ISL_427684_2020_03_23
hCoV_19_Australia_NSW201_2020_EPI_ISL_427710_2020_03_21
hCoV_19_Australia_NSW202_2020_EPI_ISL_427762_2020_03_22
hCoV_19_Australia_NSW203_2020_EPI_ISL_427765_2020_03_22
hCoV_19_Australia_NSW204_2020_EPI_ISL_427792_2020_03_25
hCoV_19_Australia_NSW205_2020_EPI_ISL_427745_2020_

hCoV_19_Australia_NSW334_2020_EPI_ISL_451596_2020_03_23
hCoV_19_Australia_NSW335_2020_EPI_ISL_451597_2020_04_08
hCoV_19_Australia_NSW336_2020_EPI_ISL_451598_2020_03_28
hCoV_19_Australia_NSW337_2020_EPI_ISL_451599_2020_03_31
hCoV_19_Australia_NSW338_2020_EPI_ISL_451600_2020_04_05
hCoV_19_Australia_NSW339_2020_EPI_ISL_451601_2020_03_25
hCoV_19_Australia_NSW33_2020_EPI_ISL_427654_2020_03_06
hCoV_19_Australia_NSW340_2020_EPI_ISL_451602_2020_03_24
hCoV_19_Australia_NSW341_2020_EPI_ISL_451603_2020_04_01
hCoV_19_Australia_NSW342_2020_EPI_ISL_451604_2020_03_29
hCoV_19_Australia_NSW344_2020_EPI_ISL_451605_2020_04_02
hCoV_19_Australia_NSW345_2020_EPI_ISL_451606_2020_04_01
hCoV_19_Australia_NSW346_2020_EPI_ISL_451607_2020_03_30
hCoV_19_Australia_NSW347_2020_EPI_ISL_451608_2020_03_27
hCoV_19_Australia_NSW348_2020_EPI_ISL_451609_2020_03_23
hCoV_19_Australia_NSW349_2020_EPI_ISL_451610_2020_04_02
hCoV_19_Australia_NSW34_2020_EPI_ISL_427655_2020_03_11
hCoV_19_Australia_NSW350_2020_EPI_ISL_451611_2020_

hCoV_19_Australia_NSW574_2020_EPI_ISL_478686_2020_03_20
hCoV_19_Australia_NSW575_2020_EPI_ISL_478687_2020_03_20
hCoV_19_Australia_NSW576_2020_EPI_ISL_478688_2020_03_20
hCoV_19_Australia_NSW577_2020_EPI_ISL_478689_2020_03_20
hCoV_19_Australia_NSW578_2020_EPI_ISL_478690_2020_03_21
hCoV_19_Australia_NSW579_2020_EPI_ISL_478691_2020_03_22
hCoV_19_Australia_NSW57_2020_EPI_ISL_417409_2020_03_16
hCoV_19_Australia_NSW580_2020_EPI_ISL_478692_2020_03_22
hCoV_19_Australia_NSW581_2020_EPI_ISL_478693_2020_03_30
hCoV_19_Australia_NSW582_2020_EPI_ISL_478694_2020_03_14
hCoV_19_Australia_NSW583_2020_EPI_ISL_478695_2020_03_14
hCoV_19_Australia_NSW584_2020_EPI_ISL_478696_2020_03_16
hCoV_19_Australia_NSW585_2020_EPI_ISL_478697_2020_03_16
hCoV_19_Australia_NSW586_2020_EPI_ISL_478698_2020_03_16
hCoV_19_Australia_NSW587_2020_EPI_ISL_478699_2020_03_14
hCoV_19_Australia_NSW588_2020_EPI_ISL_478700_2020_03_31
hCoV_19_Australia_NSW589_2020_EPI_ISL_478701_2020_04_01
hCoV_19_Australia_NSW58_2020_EPI_ISL_417410_2020_

hCoV_19_Australia_SAP070_2020_EPI_ISL_451108_2020_03_30
hCoV_19_Australia_SAP072_2020_EPI_ISL_451109_2020_03_31
hCoV_19_Australia_SAP073_2020_EPI_ISL_451110_2020_03_31
hCoV_19_Australia_SAP074_2020_EPI_ISL_451111_2020_03_31
hCoV_19_Australia_SAP075_2020_EPI_ISL_451112_2020_03_31
hCoV_19_Australia_SAP076_2020_EPI_ISL_451113_2020_03_31
hCoV_19_Australia_SAP077_2020_EPI_ISL_451114_2020_03_31
hCoV_19_Australia_SAP079_2020_EPI_ISL_451115_2020_03_31
hCoV_19_Australia_SAP080_2020_EPI_ISL_451116_2020_03_31
hCoV_19_Australia_SAP081_2020_EPI_ISL_451117_2020_03_31
hCoV_19_Australia_SAP082_2020_EPI_ISL_451118_2020_03_31
hCoV_19_Australia_SAP085_2020_EPI_ISL_451121_2020_03_31
hCoV_19_Australia_SAP087_2020_EPI_ISL_451122_2020_03_31
hCoV_19_Australia_SAP088_2020_EPI_ISL_451123_2020_04_01
hCoV_19_Australia_SAP090_2020_EPI_ISL_451125_2020_04_01
hCoV_19_Australia_SAP091_2020_EPI_ISL_451126_2020_04_01
hCoV_19_Australia_SAP093_2020_EPI_ISL_451128_2020_04_02
hCoV_19_Australia_SAP095_2020_EPI_ISL_451129_202

...

hCoV_19_Wales_PHWC_277B3_2020_EPI_ISL_432210_2020_04_01
hCoV_19_Wales_PHWC_277D1_2020_EPI_ISL_432339_2020_03_27
hCoV_19_Wales_PHWC_2780B_2020_EPI_ISL_432322_2020_04_01
hCoV_19_Wales_PHWC_2781A_2020_EPI_ISL_432380_2020_04_01
hCoV_19_Wales_PHWC_278A1_2020_EPI_ISL_432208_2020_03_31
hCoV_19_Wales_PHWC_278B0_2020_EPI_ISL_432389_2020_04_01
hCoV_19_Wales_PHWC_27917_2020_EPI_ISL_432221_2020_04_01
hCoV_19_Wales_PHWC_27935_2020_EPI_ISL_432270_2020_04_01
hCoV_19_Wales_PHWC_27944_2020_EPI_ISL_432212_2020_04_01
hCoV_19_Wales_PHWC_27971_2020_EPI_ISL_432289_2020_04_01
hCoV_19_Wales_PHWC_2799F_2020_EPI_ISL_432350_2020_04_01
hCoV_19_Wales_PHWC_27A23_2020_EPI_ISL_432245_2020_04_01
hCoV_19_Wales_PHWC_27A32_2020_EPI_ISL_432217_2020_04_01
hCoV_19_Wales_PHWC_27A41_2020_EPI_ISL_432209_2020_04_02
hCoV_19_Wales_PHWC_27A9C_2020_EPI_ISL_432180_2020_03_30
hCoV_19_Wales_PHWC_27AAB_2020_EPI_ISL_432227_2020_03_30
hCoV_19_Wales_PHWC_27ABA_2020_EPI_ISL_432198_2020_03_30
hCoV_19_Wales_PHWC_27AD8_2020_EPI_ISL_432369_202

hCoV_19_Wales_PHWC_2A11C_2020_EPI_ISL_432274_2020_04_04
hCoV_19_Wales_PHWC_2A12B_2020_EPI_ISL_432178_2020_04_04
hCoV_19_Wales_PHWC_2A167_2020_EPI_ISL_432184_2020_04_05
hCoV_19_Wales_PHWC_2A246_2020_EPI_ISL_445541_2020_04_05
hCoV_19_Wales_PHWC_2A282_2020_EPI_ISL_445545_2020_04_06
hCoV_19_Wales_PHWC_2A2FB_2020_EPI_ISL_445552_2020_04_07
hCoV_19_Wales_PHWC_2A316_2020_EPI_ISL_445553_2020_04_06
hCoV_19_Wales_PHWC_2A3DA_2020_EPI_ISL_445564_2020_04_06
hCoV_19_Wales_PHWC_2A422_2020_EPI_ISL_445567_2020_04_06
hCoV_19_Wales_PHWC_2A49B_2020_EPI_ISL_445571_2020_04_05
hCoV_19_Wales_PHWC_2A4AA_2020_EPI_ISL_445572_2020_04_05
hCoV_19_Wales_PHWC_2A4B9_2020_EPI_ISL_445573_2020_04_06
hCoV_19_Wales_PHWC_2A55C_2020_EPI_ISL_445583_2020_04_07
hCoV_19_Wales_PHWC_2A57A_2020_EPI_ISL_445585_2020_04_07
hCoV_19_Wales_PHWC_2A5B6_2020_EPI_ISL_445589_2020_04_05
hCoV_19_Wales_PHWC_2A5D4_2020_EPI_ISL_445591_2020_04_06
hCoV_19_Wales_PHWC_2A60E_2020_EPI_ISL_445594_2020_04_06
hCoV_19_Wales_PHWC_2A61D_2020_EPI_ISL_445595_202

hCoV_19_Wales_PHWC_2C0E0_2020_EPI_ISL_474275_2020_04_04
hCoV_19_Wales_PHWC_2C1FC_2020_EPI_ISL_474291_2020_04_04
hCoV_19_Wales_PHWC_2C29F_2020_EPI_ISL_474301_2020_04_04
hCoV_19_Wales_PHWC_2C2BD_2020_EPI_ISL_474303_2020_04_04
hCoV_19_Wales_PHWC_2C639_2020_EPI_ISL_445887_2020_03_31
hCoV_19_Wales_PHWC_2C6B1_2020_EPI_ISL_445895_2020_03_31
hCoV_19_Wales_PHWC_2C7BE_2020_EPI_ISL_445910_2020_03_31
hCoV_19_Wales_PHWC_2C806_2020_EPI_ISL_445915_2020_03_31
hCoV_19_Wales_PHWC_2C815_2020_EPI_ISL_445916_2020_04_01
hCoV_19_Wales_PHWC_2CA4C_2020_EPI_ISL_445938_2020_03_31
hCoV_19_Wales_PHWC_2CAB5_2020_EPI_ISL_445945_2020_04_01
hCoV_19_Wales_PHWC_2CAC4_2020_EPI_ISL_445946_2020_04_01
hCoV_19_Wales_PHWC_2CB1C_2020_EPI_ISL_445950_2020_04_01
hCoV_19_Wales_PHWC_2D3B9_2020_EPI_ISL_445980_2020_04_04
hCoV_19_Wales_PHWC_2D6ED_2020_EPI_ISL_446014_2020_04_04
hCoV_19_Wales_PHWC_2DDAC_2020_EPI_ISL_446024_2020_04_14
hCoV_19_Wales_PHWC_2DDF7_2020_EPI_ISL_446025_2020_04_14
hCoV_19_Wales_PHWC_2DE21_2020_EPI_ISL_446027_202

hCoV_19_Wales_PHWC_34425_2020_EPI_ISL_446920_2020_04_16
hCoV_19_Wales_PHWC_34452_2020_EPI_ISL_446923_2020_04_16
hCoV_19_Wales_PHWC_3448F_2020_EPI_ISL_446926_2020_04_16
hCoV_19_Wales_PHWC_34504_2020_EPI_ISL_446934_2020_04_16
hCoV_19_Wales_PHWC_34610_2020_EPI_ISL_446949_2020_04_21
hCoV_19_Wales_PHWC_34698_2020_EPI_ISL_446957_2020_04_20
hCoV_19_Wales_PHWC_346B6_2020_EPI_ISL_446959_2020_04_20
hCoV_19_Wales_PHWC_3474A_2020_EPI_ISL_446968_2020_04_20
hCoV_19_Wales_PHWC_3480B_2020_EPI_ISL_446979_2020_04_16
hCoV_19_Wales_PHWC_348FC_2020_EPI_ISL_446992_2020_04_18
hCoV_19_Wales_PHWC_34926_2020_EPI_ISL_446995_2020_04_18
hCoV_19_Wales_PHWC_34CC3_2020_EPI_ISL_474421_2020_04_20
hCoV_19_Wales_PHWC_35776_2020_EPI_ISL_474471_2020_04_24
hCoV_19_Wales_PHWC_357EF_2020_EPI_ISL_474478_2020_04_25
hCoV_19_Wales_PHWC_35916_2020_EPI_ISL_474497_2020_04_23
hCoV_19_Wales_PHWC_35A5F_2020_EPI_ISL_474517_2020_04_24
hCoV_19_Wales_PHWC_35D83_2020_EPI_ISL_474566_2020_04_27
hCoV_19_Wales_PHWC_35EEA_2020_EPI_ISL_474588_202

hCoV_19_Wuhan_IVDC_HB_01_2019_EPI_ISL_402119_2019_12_30
hCoV_19_Wuhan_IVDC_HB_04_2020_EPI_ISL_402120_2020_01_01
hCoV_19_Wuhan_IVDC_HB_05_2019_EPI_ISL_402121_2019_12_30
hCoV_19_Wuhan_IVDC_HB_GX02_2019_EPI_ISL_434534_2019_12_30
hCoV_19_Wuhan_OS52_2020_EPI_ISL_450500_2020_01_10
hCoV_19_Wuhan_WH01_2019_EPI_ISL_406798_2019_12_26
hCoV_19_Wuhan_WH03_2020_EPI_ISL_406800_2020_01_01
hCoV_19_Wuhan_WH04_2020_EPI_ISL_406801_2020_01_05
hCoV_19_Wuhan_WHU01_2020_EPI_ISL_406716_2020_01_02
hCoV_19_Wuhan_WHU02_2020_EPI_ISL_406717_2020_01_02
hCoV_19_Wuhan_WHUH001_2020_EPI_ISL_449476_2020_01_22
hCoV_19_Wuhan_WHUH002_2020_EPI_ISL_449477_2020_01_22
hCoV_19_Wuhan_WHUH003_2020_EPI_ISL_449478_2020_01_22
hCoV_19_Wuhan_WHUH004_2020_EPI_ISL_449479_2020_01_22
hCoV_19_Wuhan_WHUH005_2020_EPI_ISL_449480_2020_01_22
hCoV_19_Wuhan_WHUH006_2020_EPI_ISL_449481_2020_01_22
hCoV_19_Wuhan_WHUH007_2020_EPI_ISL_449482_2020_01_22
hCoV_19_Wuhan_WHUH008_2020_EPI_ISL_449483_2020_01_22
hCoV_19_Wuhan_WHUH011_2020_EPI_ISL_449484_2020_0